Data Wrangling Notebook for Famoso 2017 Supplemental Data 1
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [20]:
import pandas as pd
import numpy as np
import uuid

Silence unnecessary errors

In [21]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import Famoso 2017 data

In [22]:
data = pd.read_csv("../Original_Data/Famoso_2017_Supplemental_Data_1.csv")

Process Specimen # by splitting into institutionCode, collectionCode, and catalogNumber

In [29]:
data = data.assign(institutionCode = "", collectionCode = "", catalogNumber = "")

for i in data["Specimen #"].index:
    string = str(data["Specimen #"][i]).split()
    if len(string) == 2:
        data["instutionCode"][i] = string[0]
        data["catalogNumber"][i] = string[1]
    elif len(string) == 3:
        data["instutionCode"][i] = string[0]
        data["collectionCode"][i] = string[1]
        data["catalogNumber"][i] = string[2]

SyntaxError: invalid character in identifier (<ipython-input-29-bdda9d2ce8d5>, line 10)

Add broken note to materialSampleCondition

In [ ]:
def condition_correct(note): 
    """Add broken to materialSampleCondition if found in notes column"""
    if str(note).contains("broken")
        return "broken"
    
data = data.assign("materialSampleCondition" = data['notes'].apply(condition_correction))

Process basisOfRecord using information found in Famoso et al 2017 and dataset

In [ ]:
def basis_correction(note): 
    """Add broken to basisOfRecord if found in notes column, other wise assigns FossilSpecimens"""
    if note = "cast"
        return note
    else:
        return "FossilSpecimen"
    
data = data.assign("basisOfRecord" = data['notes'].apply(basis_correction))

Process scientificName and verbatimScientificName 

In [6]:
# Create verbatimScientificName (Leave out of final dataset for now)
data = data.assign(verbatimScientificName = data['Genus'] + " " + data['species'])
 
# Create scientificName from verbatimScientificName data and fill nan with "Unknown"
data = data.assign(scientificName = data["verbatimScientificName"]).fillna("Unknown")

Adding additional required GEOME columns

In [7]:
data = data.assign(country = "USA", yearCollected = "Unknown")

Set samplingProtocol and measurementMethod 

In [ ]:
citation = "Famoso, N. A. (2017). Statistical analysis of dental variation in the oligocene EQUIDMIOHIPPUS(MAMMALIA, perissodactyla) of Oregon. Journal of Paleontology, 91(5), 1060-1068. doi:10.1017/jpa.2017.42"

data = data.assign(samplingProtocol = citation, measurementMethod = citation)

Rearrange columns so that template columns are first, followed by measurement values

In [8]:
# Create column list
cols = data.columns.tolist()

# Specify desired columns
cols = ['Specimen #',
        'Locality',
        'notes',
        'Formation',
        'scientificName',
        'samplingProtocol',
        'measurementMethod',
        'institutionCode',
        'collectionCode',
        'catalogNumber'
        'member',
        'materialSampleCondition'
        'basisOfRecord']

# Subset dataframe
data = data[cols]

Matching template and column terms

In [9]:
# Renaming columns 
#TODO: fix Specimen #
data = data.rename(columns = {'Locality':'verbatimLocality',
                              'Formation':'formation',
                              'Member':'member',
                              'geologic subunit':'bed',
                              'notes':'dynamicProperties'})

Create necessary materialSampleID column and populate with UUID (use hex to remove dashes). 

In [10]:
data = data.assign(materialSampleID = '')
data['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(data.index))]

Create a long version of the data frame

In [11]:
# Creating long version, first specifiying keep variables, then naming variable and value
longVers = pd.melt(data, 
                id_vars = ['individualID',
                           'verbatimLocality',
                           'country',
                           'yearCollected',
                           'scientificName',
                           'samplingProtocol',
                           'measurementMethod',
                           'materialSampleCondition',
                           'institutionCode',
                           'collectionCode',
                           'catalogNumber',
                           'formation',
                           'member',
                           'bed',
                           'basisOfRecord'], 
                            var_name = 'measurementType', 
                            value_name = 'measurementValue')

Create diagnosticID which is a unique number for each measurement

In [12]:
longVers = longVers.assign(diagnosticID = '')
longVers['diagnosticID'] = np.arange(len(longVers))

If measurement value equals N/a, delete entire row

In [ ]:
longVers = longVers.dropna(subset = ['measurementValue'])

Writing long data csv file

In [ ]:
longVers.to_csv('../Mapped_Data/[].csv', index = False)